In [1]:
!pip install transformers
# the following are needed for implement of BigBird
#!pip install transformers==4.18.0
#!pip install sentencepiece
#!pip install flax

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 29.8 MB/s 
     |████████████████████████████████| 895 kB 46.2 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!nvidia-smi
import torch
torch.cuda.is_available()

Thu Apr 28 05:39:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import BertTokenizer, BigBirdTokenizer
import torch
from torch import nn
from transformers import BertModel, BigBirdModel
import numpy as np
from torch.optim import Adam
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os

df = pd.read_csv("train.csv")
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...


In [4]:
labels = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6
          }
prev_label = {'Lead':0,
          'Position':1,
          'Claim':2,
          'Concluding Statement':3,
          'Evidence':4,
          'Counterclaim':5,
          'Rebuttal':6,
          'Start':7
          }

In [5]:
sequence = df['predictionstring']
sequence = sequence.apply(lambda x: x.split())
sequence = sequence.apply(lambda x: list(map(int, x)))
sequence
df_new = df
start, end = [],[]
for i in range(len(sequence)):
    start.append(sequence[i][0])
    end.append(sequence[i][-1])
df_new['Start'] = start
df_new['End'] = end

prev = [7]

for i in range(1, len(sequence)):
    if df_new['End'][i-1] < df_new['Start'][i]:
        prev.append(prev_label[df['discourse_type'][i-1]])
    else:
        prev.append(7)

In [6]:
df_new['one_hot'] = prev
df_new

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,Start,End,one_hot
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,1,44,7
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,45,59,0
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,60,75,1
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,76,138,4
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,139,162,4
...,...,...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,386,572,4
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,576,827,4
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,828,838,4
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,839,902,1


In [7]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['discourse_type']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 256, truncation=True,
                                return_tensors="pt") for text in df['discourse_text']]
        self.prev = [one_hot for one_hot in df['one_hot']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def get_batch_prev(self, idx):
        return self.prev[idx]
    
    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_prev = self.get_batch_prev(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_prev, batch_y

In [9]:
np.random.seed(112)
df_new_new = df[:70000]
df_train, df_val, df_test = np.split(df_new_new.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_new_new)), int(.9*len(df_new_new))])

print(len(df_train),len(df_val), len(df_test))

56000 7000 7000


In [10]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, prev ,train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                train_label=train_label.to(torch.int64)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                prev = torch.nn.functional.one_hot(prev, num_classes=8).to(device)

                output = model(input_id, mask, prev)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, prev_val ,val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    prev_val = torch.nn.functional.one_hot(prev_val, num_classes=8).to(device)

                    output = model(input_id, mask, prev)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 4
LR = 1e-6

#model = BertClassifier()              
#train(model, df_train, df_val, LR, EPOCHS)

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')


print(len(df[:70000]))

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

70000


# Models (Bert, BigBird, BerCnn)

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()
        
        #self.bert = BigBirdModel.from_pretrained('google/bigbird-roberta-base')
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 7)
        self.linear_2 = nn.Linear(15,7)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask, prev_identity):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        linear_output = torch.cat((linear_output, prev_identity), 1)
        linear_output = self.linear_2(linear_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [12]:
class Bert_CNN1D(nn.Module):

    def __init__(self, dropout=0.5):

        super(Bert_CNN1D, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 7)
        self.linear_1 = nn.Linear(512, 7)
        self.linear_2 = nn.Linear(15,7)
        self.relu = nn.ReLU()
        
        self.conv = nn.Conv1d(15,1,1)
        self.conv0 = nn.Conv1d(1,5,1,padding='same')
        self.conv1 = nn.Conv1d(1,5,3,padding='same')
        self.conv2 = nn.Conv1d(1,5,6,padding='same')
        self.pool = nn.MaxPool1d(2,stride=2)
        
        
    def forward(self, input_id, mask, prev_identity):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        pooled_output = self.dropout(pooled_output)
        #for param in self.bert.parameters():
        #    param.requires_grad = False
        
        linear_output = torch.reshape(pooled_output,(2,1,768))
        linear_output0 = self.conv0(linear_output)#self.pool(self.relu(self.conv0(linear_output)))
        linear_output1 = self.conv1(linear_output)#self.pool(self.relu(self.conv1(linear_output)))
        linear_output2 = self.conv2(linear_output)#self.pool(self.relu(self.conv2(linear_output)))
        linear_output = torch.cat((linear_output0,linear_output1, linear_output2),1)
        linear_output = self.conv(linear_output)
        linear_output = linear_output.view(linear_output.size(0),-1)
        
        linear_output = self.linear(linear_output)
        linear_output = torch.cat((linear_output, prev_identity), 1)
        linear_output = self.linear_2(linear_output)
        final_layer = self.relu(linear_output)

        return final_layer
    
EPOCHS = 4
LR = 1e-6
 
#model = Bert_CNN1D()
#train(model1D, df_train, df_val, LR, EPOCHS)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#model = torch.load('BERTmodel')
model.load_state_dict(torch.load('BERT'))

In [13]:
#model = torch.load('BERT_CNN_model_6,5')
model.load_state_dict(torch.load('BERT_CNN_6,5'))

<All keys matched successfully>

In [ ]:
#model = torch.load('bigbird-roberta-base_model')
model.load_state_dict(torch.load('bigbird-roberta-base'))

In [ ]:
train(model, df_train, df_val, 1e-6, 2)

  8%|▊         | 2164/28000 [08:56<1:46:45,  4.03it/s]

In [ ]:
#torch.save(model, 'BERT_model')
torch.save(model.state_dict(), 'BERT')

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.save(model, '/content/drive/MyDrive/BERT_model.pth')
torch.save(model.state_dict(), '/content/MyDrive/drive/BERT.pth')

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, prev, test_label in test_dataloader:
            
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)
            prev = torch.nn.functional.one_hot(prev, num_classes=8).to(device)
            
            output = model(input_id, mask, prev)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)